In [1]:
import pandas as pd

import sys
sys.path.append('../../')

from src.utils.preprocessing import check_negative, recursive_agg

In [2]:
df = pd.read_csv('../../data/processed/balances2022.csv',
                 dtype={'ruc': str, 'codigo': str, 'valor': float})


ctns = pd.read_csv('../../data/processed/balances2022_meta.csv',
                   dtype={'codigo': str, 'cuenta': str})

**Data Quality Checks**

Quality Checks Pipeline

In [3]:
# 1790016919001 for Corporacion Favorita
df_favorita = (df
 .query('ruc=="1790016919001"')
 .merge(
     ctns, 
     how='inner', 
     on='codigo'
 )
 # check the substracting accounts are registered with negative numbers
 .pipe(check_negative)
 # check that immidiate children accounts add up to the value of the larger accounts
 .assign(valor=lambda df_: [recursive_agg(df_, v) 
                            for v in df_.codigo.values])
)

**Aggregations and Visualization**

Aggregations and Visualization Pipeline

In [5]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(
    style="whitegrid",
    rc={"figure.figsize": (12, 8)},
)

In [28]:
df_favorita

,ruc,codigo,valor,cuenta
0,1790016919001,1,2.480404e+09,ACTIVO
1,1790016919001,101,7.016353e+08,ACTIVO CORRIENTE
2,1790016919001,10101,7.283312e+06,EFECTIVO Y EQUIVALENTES DE EFECTIVO
3,1790016919001,1010101,1.342367e+06,CAJA
4,1790016919001,1010102,0.000000e+00,INSTITUCIONES FINANCIERAS PÚBLICAS
...,...,...,...,...
617,1790016919001,80008,0.000000e+00,IMPUESTO SOBRE LAS GANANCIAS RELATIVO A OTRO R...
618,1790016919001,80009,0.000000e+00,OTROS (DETALLAR EN NOTAS)
619,1790016919001,801,0.000000e+00,RESULTADO INTEGRAL TOTAL DEL AÑO
620,1790016919001,80101,0.000000e+00,PROPIETARIOS DE LA CONTROLADORA


In [20]:
df_favorita.query("codigo=='401'")

,ruc,codigo,valor,cuenta
376,1790016919001,401,2.355580e+09,INGRESOS DE ACTIVIDADES ORDINARIAS


In [21]:
df_favorita.query("codigo=='707'")

,ruc,codigo,valor,cuenta
608,1790016919001,707,1.526791e+08,GANANCIA (PÉRDIDA) NETA DEL PERIODO


In [25]:
df_favorita.query("codigo=='501'").valor.sum()

1712662748.61

In [26]:
df_favorita.query("codigo=='401'").valor.sum()

2355580171.13

In [27]:
2355580171.13 - 1712662748.61

642917422.5200002